In [1]:
import torch
import numpy as np
import pandas as pd
import matchzoo as mz
print('matchzoo version', mz.__version__)

matchzoo version 1.2


In [2]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]
print("`ranking_task` initialized with metrics", ranking_task.metrics)

`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


In [3]:
print('data loading ...')
train_pack_raw = mz.datasets.wiki_qa.load_data('train', task=ranking_task)
dev_pack_raw = mz.datasets.wiki_qa.load_data('dev', task=ranking_task, filtered=True)
test_pack_raw = mz.datasets.wiki_qa.load_data('test', task=ranking_task, filtered=True)
print('data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`')

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [4]:
preprocessor = mz.preprocessors.BasicPreprocessor(
    truncated_length_left = 10,
    truncated_length_right = 40,
    filter_low_freq = 2
)

In [5]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 6999.39it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4044.06it/s]
Building Vocabulary from a datapack.: 100%|██████████| 404432/404432 [00:00<00:00, 2579202.84it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 7853.43it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 3797.80it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 762758.98it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 7380.50it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<00

In [6]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f66fb1e47b8>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f66fb1e4dd8>,
 'vocab_size': 16675,
 'embedding_input_dim': 16675}

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [8]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=5,
    num_neg=1,
    batch_size=20,
    resample=True,
    sort=False
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
    batch_size=20
)

In [9]:
padding_callback = mz.models.KNRM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    stage='dev',
    callback=padding_callback
)

In [10]:
model = mz.models.KNRM()

model.params['task'] = ranking_task
model.params['embedding'] = embedding_matrix
model.params['kernel_num'] = 21
model.params['sigma'] = 0.1
model.params['exact_sigma'] = 0.001

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

KNRM(
  (embedding): Embedding(16675, 100, padding_idx=0)
  (kernels): ModuleList(
    (0): GaussianKernel()
    (1): GaussianKernel()
    (2): GaussianKernel()
    (3): GaussianKernel()
    (4): GaussianKernel()
    (5): GaussianKernel()
    (6): GaussianKernel()
    (7): GaussianKernel()
    (8): GaussianKernel()
    (9): GaussianKernel()
    (10): GaussianKernel()
    (11): GaussianKernel()
    (12): GaussianKernel()
    (13): GaussianKernel()
    (14): GaussianKernel()
    (15): GaussianKernel()
    (16): GaussianKernel()
    (17): GaussianKernel()
    (18): GaussianKernel()
    (19): GaussianKernel()
    (20): GaussianKernel()
  )
  (out): Linear(in_features=21, out_features=1, bias=True)
)
Trainable params:  1667522


In [11]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [ ]:
trainer.run()

[Iter-255 Loss-0.430]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.337 - normalized_discounted_cumulative_gain@5(0.0): 0.4151 - mean_average_precision(0.0): 0.3841



[Iter-510 Loss-0.107]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3684 - normalized_discounted_cumulative_gain@5(0.0): 0.4503 - mean_average_precision(0.0): 0.4113



[Iter-765 Loss-0.040]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3331 - normalized_discounted_cumulative_gain@5(0.0): 0.4163 - mean_average_precision(0.0): 0.3916



[Iter-1020 Loss-0.017]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3386 - normalized_discounted_cumulative_gain@5(0.0): 0.4214 - mean_average_precision(0.0): 0.3854



[Iter-1275 Loss-0.014]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3097 - normalized_discounted_cumulative_gain@5(0.0): 0.3901 - mean_average_precision(0.0): 0.3551



[Iter-1530 Loss-0.012]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3381 - normalized_discounted_cumulative_gain@5(0.0): 0.4276 - mean_average_precision(0.0): 0.388



[Iter-1785 Loss-0.007]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3271 - normalized_discounted_cumulative_gain@5(0.0): 0.4058 - mean_average_precision(0.0): 0.3651



[Iter-2040 Loss-0.004]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3502 - normalized_discounted_cumulative_gain@5(0.0): 0.4255 - mean_average_precision(0.0): 0.3996



[Iter-2295 Loss-0.006]:
